<a href="https://colab.research.google.com/github/JacopoKahl/gan_mnist/blob/main/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generative Adversial Network

Fully-connected networks (generator and discriminator) as thew original implementation (2014)

Example on the MNIST dataset

##Imports

In [1]:
#Keras imports
from keras.models import Sequential #Sequential model 
from keras.layers import Dense, Dropout

from keras.layers.advanced_activations import LeakyReLU #advanced LeakyRelu activation layer

#Dataset
from keras.datasets import mnist

import numpy as np #import numpy library

#Visualization 
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

##Load data

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() #load dataset

#Normalization of the dataset
X_train = (X_train.astype(np.float32) - 127.5) /127.5 #normaliza pixel values in the [-1,1] interval
X_test = (X_test.astype(np.float32) - 127.5) /127.5 #normaliza pixel values in the [-1,1] interval

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
print(X_train.shape)

(60000, 28, 28)


##Reshape input image into 1D array

We need to feed the input to fully connected network

In [4]:
#Save original dimension before reshaping

num_Samples = X_train.shape[0]
rows = X_train.shape[1]
cols = X_train.shape[2]

X_train = X_train.reshape(num_Samples, rows*cols)
print(X_train.shape)

(60000, 784)


##Models 

###Network parameters

In [5]:
gen_InputDim = 100 #Generator input dimension (like strating from the latent space)


##Generator

The generator architecture is like a decoder in the autoencoder.

In [7]:
generator = Sequential()

generator.add(Dense(units=256, input_dim= gen_InputDim))
generator.add(LeakyReLU(0.2)) #LeakyRealu with reduced alpha (slope of the negative segment) wrt default parameter

generator.add(Dense(units=512))
generator.add(LeakyReLU(0.2)) #LeakyRealu with reduced alpha (slope of the negative segment) wrt default parameter

generator.add(Dense(units=1024))
generator.add(LeakyReLU(0.2)) #LeakyRealu with reduced alpha (slope of the negative segment) wrt default parameter

generator.add(Dense(units=X_train.shape[1], activation = 'tanh')) #tanh activation returns values in the [-1,1] interval


In [9]:
generator.compile(loss='binary_crossentropy', optimizer='adam')
generator.summary()
#plot_model(generator)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 784)              

##Discriminator 

The discriminator is an image classifier. The output is binary: 1(true) for original image nd 0 for fakes (generated by the generator).